import Basic Module

In [34]:
import os
import torch
import numpy as np
from pathlib import Path

CUDA(GPU) 확인

In [35]:
# 단일 GPU 사용
GPU_ID = 0 
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using GPU: {GPU_ID}")
print(f"Device: {DEVICE}")

# 멀티 GPU 사용 
# # GPU_IDS = [0, 1, 2, 3]  # 사용할 GPU 리스트
# os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(map(str, GPU_IDS))
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# USE_MULTI_GPU = len(GPU_IDS) > 1 and torch.cuda.device_count() > 1
# print(f"Using GPUs: {GPU_IDS}")
# print(f"Available GPU count: {torch.cuda.device_count()}")
# if USE_MULTI_GPU:
#     BATCH_SIZE = BATCH_SIZE * len(GPU_IDS)  # 멀티 GPU시 배치 크기 조정
#     print(f"Adjusted batch size for multi-GPU: {BATCH_SIZE}")
# 시드 설정 (재현가능성)

Using GPU: 0
Device: cuda


데이터셋 & 모델 리스트

In [36]:
DATASET_ROOT = "./dataset"  # 심볼릭 링크된 데이터셋 루트 폴더
DATASET_LIST = [
    'LEVIR-CD+',
    'WHU-CD',
    'CLCD',
    'CaBuAr-CD',
    'S2Looking-CD',
    'SEN1Floods11-CD'
]

MODEL_LIST = [
    'A2Net',
    'Changer',
    'Change3D',
    'STRobustNet',
    'USSFC-Net'
    # 'ChangeMamba',
    # 'CDMamba',
    # 'ChangeCLIP',
    # 'EATDER'
]

시드 설정

In [37]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

##### 고정 메트릭 설정

In [38]:
# 이미지 설정
IMG_SIZE = 256 
IN_CHANNELS = 3  # RGB
OUT_CHANNELS = 1  # Binary change detection

# 학습 설정
BATCH_SIZE = 16
NUM_WORKERS = 4
MAX_ITERATIONS = 100000  # 데이터셋 크기와 무관하게 고정


##### 모델 & 데이터셋 설정

In [45]:
# 실험할 데이터셋 선택
test_dataset = 'LEVIR-CD+'  # LEVIR-CD+
# test_dataset = DATASET_LIST[0]  # LEVIR-CD+

# 실험할 모델 선택
test_model = 'Change3D'

print(f"Test dataset: {test_dataset}")
if test_model not in MODEL_LIST:
    raise ValueError(f"Model {test_model} not in MODEL_LIST. Choose from: {MODEL_LIST}")
print(f"Test model: {test_model}")

Test dataset: LEVIR-CD+
Test model: Change3D


In [40]:
from configs import get_model_config

model_config = get_model_config(test_model)

optimizer = model_config['optimizer']
learning_rate = model_config['learning_rate']
weight_decay = model_config['weight_decay']
betas = model_config['betas']
eps = model_config['eps']
scheduler = model_config['scheduler']
momentum = model_config['momentum']

print(f"Model configurations:")
print(f"  Optimizer: {optimizer}")
print(f"  Learning rate: {learning_rate}")
print(f"  Weight decay: {weight_decay}")
if betas:
    print(f"  Betas: {betas}")
if momentum:
    print(f"  Momentum: {momentum}")
print(f"  Scheduler: {scheduler}")


Model configurations:
  Optimizer: adam
  Learning rate: 0.0002
  Weight decay: 0.0001
  Betas: (0.9, 0.999)
  Scheduler: poly


실험 폴더 설정

In [41]:
test_path = f"experiments/{test_dataset}"
test_dir = Path(f"{test_path}")
test_dir.mkdir(parents=True, exist_ok=True)

model_dir = Path(f"{test_path}/{test_model}")
model_dir.mkdir(parents=True, exist_ok=True)

checkpoint_dir = model_dir / "checkpoints"
checkpoint_dir.mkdir(exist_ok=True)

print(f"Experiment directory: {model_dir}")

Experiment directory: experiments/WHU-CD/Change3D


Iteration 계산 
MAX_ITERATIONS = 100000을 기준으로 epoch 수 계산


In [44]:
dataset_path = Path(DATASET_ROOT) / test_dataset
print(dataset_path)
if dataset_path.exists():
    print(f"Dataset found: {dataset_path}")
    splits = ['train', 'val', 'test']
    dataset_info = {}
    
    for split in splits:
        split_path = dataset_path / split
        if split_path.exists():
            img_count = len(list((split_path / 't1').glob('*')))
            dataset_info[split] = img_count
            print(f"  {split}: {img_count} images")
    
    # Epoch 수 계산 (MAX_ITERATIONS 기준)
    if 'train' in dataset_info:
        train_samples = dataset_info['train']
        iterations_per_epoch = train_samples // BATCH_SIZE
        EPOCHS = MAX_ITERATIONS // iterations_per_epoch
        
        print(f"\nTraining iterations info:")
        print(f"  Train samples: {train_samples}")
        print(f"  Iterations per epoch: {iterations_per_epoch}")
        print(f"  Total epochs: {EPOCHS}")
        print(f"  Total iterations: {EPOCHS * iterations_per_epoch}")
else:
    print(f"Dataset not found: {dataset_path}")
    raise FileNotFoundError(f"Dataset {test_dataset} not found at {dataset_path}")

dataset/WHU-CD
Dataset found: dataset/WHU-CD
  train: 4233 images
  val: 747 images
  test: 2700 images

Training iterations info:
  Train samples: 4233
  Iterations per epoch: 264
  Total epochs: 378
  Total iterations: 99792


In [ ]:
def get_optimizer(model, config):
    """모델 설정에 따른 옵티마이저 생성"""
    if config['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(
            model.parameters(), 
            lr=config['learning_rate'],
            betas=config['betas'],
            eps=config['eps'],
            weight_decay=config['weight_decay']
        )
    elif config['optimizer'] == 'adamw':
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config['learning_rate'],
            betas=config['betas'],
            eps=config['eps'],
            weight_decay=config['weight_decay']
        )
    elif config['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(
            model.parameters(),
            lr=config['learning_rate'],
            momentum=config['momentum'],
            weight_decay=config['weight_decay']
        )
    else:
        raise ValueError(f"Unknown optimizer: {config['optimizer']}")
    
    return optimizer